In [ ]:
%autosave 0

In [ ]:
from info_nas.datasets.networks.pretrained import pretrain_network_cifar
from info_nas.datasets.networks.utils import load_nasbench
from nasbench_pytorch.datasets.cifar10 import prepare_dataset

In [ ]:
seed = 42
batch_size = 32
num_workers = 2

dataset = prepare_dataset(batch_size, root='../data/cifar/', validation_size=1000, random_state=seed,
                          num_workers=num_workers)
train, n_train, val, n_val, test, n_test = dataset

In [ ]:
import os

checkpoint_dir = '../data/train_checkpoints//'

os.listdir(f'{checkpoint_dir}')

In [ ]:
from nasbench import api

nasbench_path = '../data/nasbench_only108.tfrecord'
nb = api.NASBench(nasbench_path)

In [ ]:
import torch
from info_nas.datasets.networks.utils import load_trained_net

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

net_path = f'{checkpoint_dir}083a8045a46ac2a25a34c8fab47f6ecb.tar'
net_hash, net, info = load_trained_net(net_path, nb, device=device)
net

In [ ]:
info

In [ ]:
net.classifier.bias

- heatmaps todo
  - all net weights for e.g. airplane
  - minmaxscale to (-1,1), compare net
    - bias?
  - standardscale instead of normalize
  - predict one image, sort values by weights
    - how to normalize??

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook

labels = "airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck".split(', ')

weights = net.classifier.weight.detach().cpu()

plt.figure(figsize=(5,6))
sns.heatmap(weights, yticklabels=labels)
#plt.imshow(net.classifier.weight.detach().cpu())
plt.tight_layout()
plt.savefig('cifar_weights.png')
plt.show()

In [ ]:
import numpy as np

print(net_hash)
np.array([net_hash, net_hash, net_hash])

In [ ]:
type(net_hash)

In [ ]:
for batch_idx, (inputs, targets) in enumerate(val):
    inputs, targets = inputs.to(device), targets.to(device)
    break
    
print(inputs.shape)
print(targets.shape)

In [ ]:
nasbench = load_nasbench(nasbench_path)

In [ ]:
from nasbench_pytorch.model import Network as NBNetwork

net = nasbench[0]
net = NBNetwork((net[2], net[1]), 10)
net = net.to(device)

In [ ]:
with torch.no_grad():
    outputs = net(inputs.to(device))
    
outputs.shape

In [ ]:
with torch.no_grad():
    out_list = net.get_cell_outputs(inputs, return_inputs=False)
    
[print(o.shape) for o in out_list]
print()

In [ ]:
with torch.no_grad():
    in_list, out_list = net.get_cell_outputs(inputs, return_inputs=True)
    
[print(i.shape, ' -> ', o.shape) for i, o in zip(in_list, out_list)]
print()

In [ ]:
torch.cat(in_list[:2]).shape

In [ ]:
with torch.no_grad():
    in_list, out_list = net.get_cell_outputs(inputs, return_inputs=True)
    
for in_image in in_list[-2][0][:10]:
    print(in_image.shape)

In [ ]:
means = np.array([0.4914, 0.4822, 0.4465])[:, np.newaxis, np.newaxis]
stds = np.array([0.2023, 0.1994, 0.2010])[:, np.newaxis, np.newaxis]
im = inputs[1].detach().cpu() * stds + means

In [ ]:
targets[1]

In [ ]:
plt.figure(figsize=(1,1))
plt.imshow(im.moveaxis(0,2))
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook

for in_image in in_list[-2][1][:10]:
    plt.figure(figsize=(1,1))
    plt.imshow(in_image.detach().cpu())
    plt.show()


In [ ]:
# Check if all shapes the same

In [ ]:
for i, next_net in enumerate(nasbench[:150]):
    if (i % 5) == 0:
        print(i)

    next_net = NBNetwork((next_net[2], next_net[1]), 10)
    
    with torch.no_grad():        
        n_in, n_out = next_net.get_cell_outputs(inputs, return_inputs=True)
        
        for i, ni in zip(in_list, n_in):
            assert i.shape == ni.shape
            
        for o, no in zip(out_list, n_out):
            assert o.shape == no.shape

In [ ]:
len(nasbench)